In [4]:
from google.colab import drive
drive.mount('/content/drive')


# !unzip -qq data.zip

/content/drive/MyDrive/$03DSCoding/$75code_mipt_course_ML/homeworks/lab02_deeplearn


In [5]:
%cd '/content/drive/MyDrive/$03DSCoding/$75code_mipt_course_ML/homeworks/lab02_deeplearn'
%ls

 data/
 hw_var_1/
 Lab2_DL_part1_warmup.ipynb
 Lab2_DL_part2_overfitting.ipynb
 Lab2_DL_part2_overfitting_shalvinskiy.ipynb
 Lab2_DL_part3_poetry.ipynb
 Lab2_DL_part3_poetry_shalvinskiy.ipynb
 Lab2_DL_part5_optional.ipynb
 Lab2_DL_parts_4_and_5_optional.ipynb
 lstm
 lstm_model
 README.md
 sonnets.txt
'UCI HAR Dataset.zip'


In [20]:
%ls './data/train/2'

In [6]:
#check how bidirectional lstm works
from keras import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional
model = Sequential()
model.add(Bidirectional(LSTM(units=20, input_dim=10, use_bias=False, input_shape=(1,10))))
model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

model.build((1, 10, 10))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional (Bidirectiona  (1, 40)                  4800      
 l)                                                              
                                                                 
Total params: 4,800
Trainable params: 4,800
Non-trainable params: 0
_________________________________________________________________


### Part 5. Dogs classification (2+ points)
__Disclaimer__: Это опциональная часть задания. Здесь придется экспериментировать, подбирать оптимальную структуру сети для решения задачи и активно искать подскзаки в сети.

Предлагаем вам решить задачу классификации пород собак. Вы можете обучить сеть с нуля или же воспользоваться методом fine-tuning'а. Полезная ссылка на [предобученные модели](https://pytorch.org/docs/stable/torchvision/models.html).
Ссылка изменилась, вот новая -- https://pytorch.org/vision/stable/models.html.

Данные можно скачать [отсюда](https://www.dropbox.com/s/vgqpz2f1lolxmlv/data.zip?dl=0). Датасет представлен 50 классами пород собак, которые можно найти в папке train в соответствующих директориях. При сдаче данной части задания вместе с ноутбуком необходимо отправить .csv-файл с предсказаниями классов тестовой выборки в формате: <имя изображения>,<метка класса> по одному объекту на строку. Ниже приведите код ваших экспериментов и короткий вывод по их результатам.

Будут оцениваться качество классификации (accuracy) на тестовой выборке (2 балла) и проведенные эксперименты (1 балл).
Разбалловка следующая:
* $>=$93% - 2 points
* $>=$84% - 1.5 points
* $>=$70% - 0.75 points

In [7]:
import torch
from torchvision import datasets, transforms, models
import torch.nn.functional as F
from torch import nn
from torch import optim
from torchvision.io import read_image
from torch.utils.data import DataLoader, random_split

import torchvision.transforms as T

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from torchsummary import summary
from IPython.display import clear_output

import os
torch.manual_seed(42)

In [ ]:
#funciton for renaming test photos
def renaming():
    path = r'./data/test/'
    
    for file in os.listdir(path):
        os.makedirs(os.path.dirname(path+file.replace('.jpeg', '/')))
        os.rename(path + file, path + file.replace('.jpeg', '') + '/' + file)
renaming()

In [16]:
train_transform = transforms.Compose([transforms.RandomResizedCrop(336),
                                    transforms.RandomHorizontalFlip(),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
trainset = datasets.ImageFolder('data/train/', transform=train_transform)
len(trainset)

FileNotFoundError: ignored

In [ ]:
#ONLY FOR TESTING
test_transform = transforms.Compose([transforms.RandomResizedCrop(336),
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
testset = datasets.ImageFolder('data/test/', transform=test_transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=True)
pic_to_label = testset.class_to_idx
label_to_pic = {pic_to_label[pic] : pic for pic in pic_to_label}

In [ ]:
class_to_label = trainset.class_to_idx
label_to_class = {class_to_label[cl] : cl for cl in class_to_label}
num_classes = len(class_to_label)
num_classes

In [ ]:
trainset_size = int(len(trainset) * 0.95)
validset_size = len(trainset) - trainset_size
trainset, validset = random_split(trainset, [trainset_size, validset_size])

len(trainset)

In [ ]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
validloader = torch.utils.data.DataLoader(validset, batch_size=32, shuffle=True)

In [ ]:
for i in trainloader:
    images, labels = i
    print(labels[0])
    img = images[0].numpy().transpose(1,2,0)
    
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    img = std * img + mean
    img = np.clip(img, 0, 1)
    plt.imshow(img)
    break

In [ ]:
def validate(model, validloader, criterion, is_cuda):
    val_loss = 0
    correct = 0
    total = 0

    model.eval()
    for batch_idx, (data, target) in enumerate(tqdm(validloader)):
        if is_cuda:
            data, target = data.cuda(), target.cuda()

        output = model(data)
        loss = criterion(output, target)
        
        val_loss = val_loss + ((1 / (batch_idx + 1)) * (loss.data - val_loss))
        pred = output.data.max(1, keepdim=True)[1]
        correct += np.sum(np.squeeze(pred.eq(target.data.view_as(pred))).cpu().numpy())
        total += data.size(0)
            
    print(f'\n Validation loss: {val_loss}. Accuracy: {correct / total}({correct} out of {total})')

    return val_loss

In [ ]:
def val_train_loop(model, testloader, validloader, optimizer, criterion, n_epochs, is_cuda, save_path):
    min_valid_loss = np.inf
    period=4
    loss_history = []
    valid_loss_history = []

    for epoch in range(n_epochs):
        train_loss = 0
        valid_loss = 0
        
        model.train()
        for batch_idx, (data, target) in enumerate(tqdm(trainloader)):
            if is_cuda:
                data, target = data.cuda(), target.cuda()
            
            output = model(data)
            loss = criterion(output, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.data - train_loss))
            
        if(epoch % period == 0):
          valid_loss = validate(model, validloader, criterion, is_cuda)
          valid_loss_history.append(valid_loss.cpu().numpy())

          #save the model if validation loss has decreased
          if valid_loss < min_valid_loss:
              print(f'\nValidation loss decreased ({min_valid_loss} --> {valid_loss}). Model saved')
              torch.save(model.state_dict(), save_path)
              min_valid_loss = valid_loss
        
        loss_history.append(train_loss)
        
        clear_output(wait=True)
        plt.figure(figsize=(8, 5))
        plt.title(f"Training/validating loss")
        plt.xlabel("#epoch")
        plt.ylabel("Loss")
        plt.plot(loss_history, 'blue', label='train')
        plt.scatter(np.arange(len(valid_loss_history)) * period, valid_loss_history, c ='orange', s=100, label='validate')
        plt.legend()
        plt.show()

        print(f'\nEpoch: {epoch}. Training loss: {train_loss}. Last valloss: {min_valid_loss}')
    
    return model

In [ ]:
vgg16 = models.vgg16(pretrained=True)

for param in vgg16.features.parameters():
    param.requires_grad = False
    
if torch.cuda.is_available():
    vgg16 = vgg16.cuda()

In [ ]:
print(vgg16)

In [ ]:
vgg16.classifier[6].out_features = num_classes

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.classifier.parameters(), lr=0.001, momentum=0.9)
n_epochs = 13

In [ ]:
%%time
vgg16_trained = val_train_loop(vgg16, trainloader, validloader, optimizer, criterion, n_epochs, torch.cuda.is_available(), 'vgg16_no_tune.pt')

In [ ]:
print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [ ]:
reload_model = models.vgg16()
reload_model.classifier[6].out_features = num_classes  
reload_model.load_state_dict(torch.load('vgg16_no_tune.pt'))

if torch.cuda.is_available():
    reload_model = reload_model.cuda()

In [ ]:
validate(reload_model, validloader, criterion, torch.cuda.is_available())

In [ ]:
def test(model, testloader, is_cuda, file_name):
    model.eval()
    targets = np.array([])
    preds = np.array([])

    for (data, target) in tqdm(testloader):
        if is_cuda:
            data, target = data.cuda(), target.cuda()

        output = model(data)
        pred = output.data.max(1, keepdim=True)[1]

        targets = np.concatenate((targets, target.cpu().numpy().ravel()))
        preds = np.concatenate((preds, pred.cpu().numpy().ravel()))

    df = pd.DataFrame(data={'targets': np.int64(targets), 'preds': np.int64(preds)})
    df['targets'] = df['targets'].apply(lambda x: str(label_to_pic[x]) + '.jpeg')
    df['preds'] = df['preds'].apply(lambda x: label_to_class[x])
    df['nums'] = df['targets'].apply(lambda x: int(x.replace('.jpeg', '')))
    df.sort_values('nums').to_csv(file_name)
    return df

df = test(reload_model, testloader, torch.cuda.is_available(), 'vgg16_no_tune_resuls.csv')

In [ ]:
df.sort_values('nums').head(6)

### New train 

In [ ]:
vgg16 = models.vgg16(pretrained=True)

for param in vgg16.features.parameters():
    param.requires_grad = False
    
if torch.cuda.is_available():
    vgg16 = vgg16.cuda()

In [ ]:
vgg16.classifier[6].out_features = num_classes

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.classifier.parameters(), lr=0.001, momentum=0.9)
n_epochs = 9

In [ ]:
%%time
vgg16_trained = val_train_loop(vgg16, trainloader, validloader, optimizer, criterion, n_epochs, torch.cuda.is_available(), 'vgg16_no_tune_2.pt')

Продолжим тренировку модели

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.classifier.parameters(), lr=0.001, momentum=0.7)
n_epochs = 5

In [ ]:
%%time
vgg16_trained = val_train_loop(vgg16_trained, trainloader, validloader, optimizer, criterion, n_epochs, torch.cuda.is_available(), 'vgg16_no_tune_2.pt')

In [ ]:
df = test(vgg16_trained, testloader, torch.cuda.is_available(), 'vgg16_no_tune_2_results.csv')
df.sort_values('nums').head(6)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.classifier.parameters(), lr=0.0001, momentum=0.7)
n_epochs = 1

In [ ]:
%%time
vgg16_trained = val_train_loop(vgg16_trained, trainloader, validloader, optimizer, criterion, n_epochs, torch.cuda.is_available(), 'vgg16_no_tune_2.pt')

**Вывод** Пока что это лучший результат, также была предпринята попфтка дообучения модели птуем угулюления головы(добавил несколько linear, relu, dropout слоев, получилось хуже, обучение шло еще медленнее, loss был огромным. Также за эту работу сравнил adam и sgd отпимизаторы, осознал, насколько важно правильно подбирать параметры lr, momentum, в итоге получилось достичь точности в 82% на валидационной выборке

Еще одна попытка, будем дообучать resnet

(0): Linear(in_features=25088, out_features=4096, bias=True)

(1): ReLU(inplace=True)

(2): Dropout(p=0.5, inplace=False)

(3): Linear(in_features=4096, out_features=4096, bias=True)

(4): ReLU(inplace=True)

(5): Dropout(p=0.5, inplace=False)

(6): Linear(in_features=4096, out_features=1000, bias=True)

In [ ]:
resnet18 = models.resnet18(pretrained=True)
resnet18

In [ ]:
for param in resnet18.parameters():
    param.requires_grad = False
  
classifier = nn.Sequential(nn.Linear(512, 1024), 
                           nn.ReLU(),
                           nn.Dropout(p=0.5),
                           nn.Linear(1024, 2048), 
                           nn.ReLU(),
                           nn.Dropout(p=0.5), 
                           nn.Linear(2048, num_classes)
                           )

resnet18.fc = classifier

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(resnet18.fc.parameters(), lr=0.001, momentum=0.7)
n_epochs = 5

In [ ]:
%%time
resnet18_trained = val_train_loop(resnet18, trainloader, validloader, optimizer, criterion, n_epochs, torch.cuda.is_available(), 'resnet18.pt')